## 2.2) Setting up the foreground system: foreground database

For modeling a foreground system, we need to create a new database that is linked to ecoinvent (and biosphere) database(s) via exchanges. The simplest way to do this consists of using an excel file with a predefined template. This notebook contains the required steps to import the foreground datatabase of a pavement system that comes originally as an excel file.

In [80]:
from pathlib import Path
from pprint import pprint

In [81]:
## Import BW25 packages
import bw2data as bd
import bw2io as bi

In [82]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name

In [83]:
bd.projects.set_current(project_name)
print(project_name)
bd.databases

2024_STI_BW25_Lecture


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [84]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

## Transform the data from xlsx format to Brightway format

### Extract foreground inventory data

In [86]:
# Include the path to the foreground database (excel file)
FG_DB = Path("./Foreground.xlsx")

# Import your LCI
lci = bi.ExcelImporter(FG_DB)

Extracted 1 worksheets in 16.44 seconds


### Match the foreground database to itself

In [87]:
# Need to match FG_DB to itself
lci.match_database(fields=["name", "unit", "location"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the background db

In [88]:
# Need to match FG_DB to the biosphere
lci.match_database(ei_name, fields=["name", "reference product", "location", "unit"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the biosphere3 database

In [89]:
# Need to match FG_DB to the biosphere
lci.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields


In [90]:
bi.create_core_migrations()
# Once your package is imported we need to apply strategies
lci.apply_strategies()

# We need to match databases - name and categories but ATTENTION! the categories in
# the excel file is "None" so we will lci.write_excel() also need to match against unit.

lci.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 5.09 seconds
39 datasets
173 exchanges
0 unlinked exchanges
  


(39, 173, 0)

In [91]:
import_statistics = lci.statistics(print_stats=False)

# If all exchanges have been linked, proceed to write the database
if import_statistics[2] == 0:
    lci.write_database()
else:
    print(f"There are unlinked exchanges {import_statistics}")
    lci.write_excel()
    for unlinked_exchange in lci.unlinked:
        pprint(unlinked_exchange)

bd.databases

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|█████████████████████████████████████████| 39/39 [00:00<00:00, 2373.72it/s]

Vacuuming database 


Created database: asphalt


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

### DB structure checks

In [92]:
import bw2analyzer as bwa

Let's define and assign the name of the foreground database to a variable called "fg_name".
The name must be equal to the content of the cell B2 in the excel file "Foreground.xlsx"

In [93]:
fg_name = "asphalt"

There are many properties and functions associated with this database object. To access those, let's assign the database to a variable

In [94]:
fgdb = bd.Database(fg_name)

Let's explore the foreground db with the function "random"

In [95]:
fgdb.random()

'AC Bin, production' (kilogram, NL, None)

In [96]:
random_fg_activity = fgdb.random()
random_fg_activity

'AC Surf, 30% RAP, mofified bitumen' (kilogram, NL, None)

We can get the type of the object that was returned from the database:

In [97]:
type(random_fg_activity)

bw2data.backends.proxies.Activity

The type is an activity proxy. Activity proxies allow us to interact with the content of the database.

To see what the activity contains, we can convert it to a dictionary:

In [98]:
random_fg_activity.as_dict()

{'reference product': 'asphalt',
 'code': 'asphalt_AC_Surf_2',
 'location': 'NL',
 'amount': 1,
 'unit': 'kilogram',
 'original_ConversionDem2FU': 1,
 'name': 'AC Surf, 30% RAP, mofified bitumen',
 'worksheet name': 'LCI',
 'database': 'asphalt',
 'id': 25978}

We can get the value of each item by specifying its key. Example: let's get the value corresponding to the key 'location'

In [99]:
random_fg_activity['location']

'NL'

If we are interested in a specific activity we can use the method 'get' with the code of the activity we are interested in. For example, let's select the activity 'pavement_structure_1':

In [100]:
pavement = fgdb.get('pavement_structure_1')

In [101]:
pavement

'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)

We can explore the supply chain of this activity by using the function 'print_recursive_supply_chain'. The parameter 'max_level' allow us to define how far we want to go (i.e., depth) in the supply chain. For instance, if max_level=1, we will get the activities that are imediately linked to the activate 'pavement_structure'.

This function also supports:

- Using a custom string instead of tabs for indentation
- Writing to a file-like object instead of printing to stdout
- Rescaling all amounts to an arbitrary value
- Using a cutoff to limit what is returned


In [102]:
bwa.print_recursive_supply_chain(pavement, max_level=1)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)


If max_level=2, we will also get the activities that are linked to those listed out above.

In [103]:
bwa.print_recursive_supply_chain(pavement, max_level=2)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant, 30% RAP, regular bitumen' (kilogram, NL, None)
    2.01e+06: 'AC Surf, 30% RAP, production' (kilogram, NL, None)
    2.01e+06: 'asphalt, transport to site' (kilogram, NL, None)
    2.01e+06: 'asphalt, construction' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
    2.72e+06: 'AC Bin, materials and transport to plant' (kilogram, NL, None)
    2.72e+06: 'AC Bin, production' (kilogram, NL, None)
    2.72e+06: 'asphalt, transport to site' (kilogram, NL, None)
    2.72e+06: 'asphalt, construction' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)
    1.35e+07: 'AC Bin, materials and transport to plant' (kilogram, NL, None)
    1.35e+07: 'AC Bin, production' (kilogram, NL, None)
    1.35e+07: 'asphalt, transport to site' (kilogram, NL, None)
    1.35e+07: 'as

Similar for activities up to level 3...

In [104]:
bwa.print_recursive_supply_chain(pavement, max_level=3)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant, 30% RAP, regular bitumen' (kilogram, NL, None)
      9.24e+04: 'bitumen, production and transport' (kilogram, NL, None)
      5.91e+05: 'asphalt granulate, free of burden' (kilogram, NL, None)
      7.35e+05: 'crushed stone, production and transport' (kilogram, NL, None)
      5.18e+05: 'crushed sand, production and transport' (kilogram, NL, None)
      1.81e+04: 'other fillers, production and transport' (kilogram, NL, None)
      5.42e+04: 'weak filler' (kilogram, NL, None)
    2.01e+06: 'AC Surf, 30% RAP, production' (kilogram, NL, None)
      1.18e+04: 'market for electricity, low voltage' (kilowatt hour, NL, None)
      5.84e+05: 'heat production, natural gas, at industrial furnace >100kW' (megajoule, Europe without Switzerland, None)
      8.63e+03: 'diesel, burned in building

In [105]:
bwa.print_recursive_supply_chain(pavement, max_level=4)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant, 30% RAP, regular bitumen' (kilogram, NL, None)
      9.24e+04: 'bitumen, production and transport' (kilogram, NL, None)
        9.24e+04: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
        2.31e+04: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
      5.91e+05: 'asphalt granulate, free of burden' (kilogram, NL, None)
      7.35e+05: 'crushed stone, production and transport' (kilogram, NL, None)
        7.35e+05: 'gravel production, crushed' (kilogram, RoW, None)
        1.84e+04: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
        4.85e+05: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)
      5.18e+05: 'crushed sand, production and transport' (kilogram, NL,

Below a silly example of most of the options mentioned before:

In [106]:
bwa.print_recursive_supply_chain(pavement, max_level=5, cutoff=0.02, tab_character="🐶", amount=2)

2: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
🐶4.02e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
🐶🐶4.02e+06: 'AC Surf, materials and transport to plant, 30% RAP, regular bitumen' (kilogram, NL, None)
🐶🐶🐶1.85e+05: 'bitumen, production and transport' (kilogram, NL, None)
🐶🐶🐶🐶1.85e+05: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
🐶🐶🐶🐶🐶2.21e+05: 'market group for heat, district or industrial, natural gas' (megajoule, RER, None)
🐶🐶🐶🐶🐶1.22e+03: 'market for pitch' (kilogram, CH, None)
🐶🐶🐶🐶🐶1.85e+05: 'market for pitch' (kilogram, Europe without Switzerland, None)
🐶🐶🐶🐶4.62e+04: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
🐶🐶🐶🐶🐶1.8e+04: 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspecified' (ton kilo
🐶🐶🐶🐶🐶6.39e+03: 'transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified' (ton kil
🐶🐶🐶🐶🐶1.13e